## DSPy train/eval

Note: I use my own fork of DSPy because I had to implement asynchronous batching: https://github.com/rayanehmi/dspy/tree/feat/async_batching

In [ ]:
from pathlib import Path
from typing import Literal
import os 

DATA_PATH = Path.cwd().parent / "data"
DATA_TYPE : Literal["train", "rank", "final"] = "train"
OUTPUT_DIR = os.path.join(DATA_PATH)


In [ ]:
# Load complete data
import polars as pl

SEGMENTS_PATH = os.path.join(OUTPUT_DIR, "llm_segments_" + DATA_TYPE + ".parquet")
df = pl.read_parquet(SEGMENTS_PATH)
df.head()

In [ ]:
import dspy
from dotenv import load_dotenv
load_dotenv()

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
gpt_4_1 = dspy.LM("openai/gpt-4.1", api_key=api_key)
gpt_4_1_nano = dspy.LM("openai/gpt-4.1-nano", api_key=api_key)
groq_api_key = os.getenv("GROQ_API_KEY")
gpt_oss_120b = dspy.LM("groq/openai/gpt-oss-120b", api_key=groq_api_key, cache=False)
gpt_5_1_instant = dspy.LM("openai/gpt-5.1", api_key=api_key, temperature=1.0, max_tokens=32000, reasoning_effort="none", cache=False)

json_adapter = dspy.JSONAdapter()

dspy.configure(lm=gpt_oss_120b, adapter=json_adapter)

In [52]:
from typing import Any

class BurntFuelPrediction(dspy.Signature):
    """Predict the amount of fuel burnt in kgs by the plane over the given segment of flight.
    Segment data is constructed from noisy telemetry: use your common sense if values seem wrong.
    hint: vertical_rate_balance contains positive_frac, negative_frac and near_zero_frac, each corresponding 
    to the share of samples where vertical rate is respectively more than, less than or around 64 ft/min.
    hint 2: estimate the fuel weight penalty (heavy in the beginning, lighter in the end).
    """
    features : dict[str, Any] = dspy.InputField()
    fuel_kg : float = dspy.OutputField() 

# Zero-shot chain of thought
fuel_cot = dspy.ChainOfThought(BurntFuelPrediction)

In [35]:
def df_to_dspy_examples(row, with_fuel: bool = True):
    """Converts a row to a dspy.Example."""
    row_data = row.to_dicts()[0] if hasattr(row, "to_dicts") else row

    def clean(value):
        return "" if value is None else value

    inputs = [
        "aircraft_type",
        "origin_name",
        "origin_destination",
        "track_points_compact",
        "track_points_compact",
        "vertical_rate_balance"
    ]
    
    features = {key: clean(row_data.get(key)) for key in inputs}
    example = dspy.Example(features=features).with_inputs("features")
    if with_fuel:
        example.fuel_kg = clean(row_data.get("fuel_kg"))
    return example


In [36]:
examples = [
    df_to_dspy_examples(row, with_fuel=True)
    for row in df.iter_rows(named=True)
]
examples[0]

Example({'features': {'aircraft_type': 'B789', 'origin_name': 'Kuala Lumpur International Airport', 'origin_destination': '', 'track_points_compact': 'time 2025-04-13T02:31:04.447000->2025-04-13T03:01:04.487000 (30 min) | sources acars:2, adsb:3076 | altitude 3.597e+04 -> 3.597e+04 -> 3.6e+04 -> 3.6e+04 (delta 24.99, range 24.99, mean 3.599e+04) | groundspeed 467 -> 474 -> 471 -> 476 (delta 9, range 15, mean 472) | vertical_rate 0 -> 0 -> 0 -> -64 (delta -64, range 128, mean -3.36) | mach 0.86 -> 0.86 (delta 0, range 0, mean 0.86) | path 45.18/24.35 -> 45.9/22.72 -> 46.62/20.97 -> 47.22/19.52 | delta_lat 2.033 delta_lon -4.833 | phase cruise | vr balance +0.00 / -0.00 / ~0 1.00', 'vertical_rate_balance': {'positive_frac': 0.0, 'negative_frac': 0.0, 'near_zero_frac': 1.0}}, 'fuel_kg': 2500.0}) (input_keys={'features'})

In [37]:
import random
import copy

randomized_examples = copy.deepcopy(examples)
random.Random(42).shuffle(randomized_examples)

## Metrics

In [38]:
import math

def float_metric(gold: dspy.Example, pred: dspy.Prediction, trace=None):
    """Return a scalar score (negative squared error) for the evaluator."""
    true_value = gold.fuel_kg
    pred_value = pred.fuel_kg
    if true_value is None or pred_value is None:
        return float("nan")

    squared_error = (true_value - pred_value) ** 2

    if trace is None: # if we're doing evaluation or optimization
        return -squared_error
    else:  # During bootstrapping / trace collection we simply mark good demos.
        return squared_error < 40000  # Squared error 200.
    
fake_example = dspy.Example(features={"foo": "bar"}, fuel_kg=500.0)
fake_prediction = dspy.Prediction(features={"foo": "bar"}, fuel_kg=600.0)
print(float_metric(fake_example, fake_prediction))  # error -100.0
print(float_metric(fake_example, fake_prediction, trace='foo'))  # True

-10000.0
True


## Evaluation

In [39]:
from dspy.evaluate import Evaluate

In [47]:
evaluator_50 = Evaluate(
    devset=randomized_examples[:50],
    num_threads=50, 
    display_progress=True, 
    display_table=True
)

evaluator_500 = Evaluate(
    devset=randomized_examples[:400],
    num_threads=15, 
    display_progress=True, 
    display_table=True
)

In [48]:
with dspy.context(lm=gpt_oss_120b):
    eval_results = evaluator_500(fuel_cot, metric=float_metric)

Average Metric: -26070998.89 / 231 (-11286146.7%):  57%|█████▊    | 230/400 [00:20<00:48,  3.48it/s]

2025/11/29 22:00:32 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: -26091407.26 / 232 (-11246296.2%):  58%|█████▊    | 232/400 [00:20<00:32,  5.22it/s]

2025/11/29 22:00:33 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/11/29 22:00:33 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/11/29 22:00:33 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/11/29 22:00:33 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: -26186239.29 / 235 (-11143080.5%):  59%|█████▉    | 235/400 [00:21<00:36,  4.51it/s]

...

Average Metric: -86106547.58 / 382 (-22540981.0%):  97%|█████████▋| 389/400 [00:50<00:01,  5.50it/s]

2025/11/29 22:01:03 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: -86597409.44 / 393 (-22034964.2%): 100%|██████████| 400/400 [00:54<00:00,  7.37it/s]

2025/11/29 22:01:06 INFO dspy.evaluate.evaluate: Average Metric: -86597409.4413901 / 400 (-21649352.4%)


,features,example_fuel_kg,pred_fuel_kg,reasoning,float_metric,fuel_kg
0,"{'aircraft_type': 'A320', 'origin_name': 'Guatemala City La Aurora...",54.000000,78.2,The segment lasts about 3.13 minutes (from 05:28:03 to 05:31:11). ...,✔️ [-585.640],NaN
1,"{'aircraft_type': 'A21N', 'origin_name': 'Guadalajara Internationa...",300.000000,235.0,The segment lasts roughly 10 minutes (9.99 min) and is a descent f...,✔️ [-4225.000],NaN
2,"{'aircraft_type': 'A20N', 'origin_name': 'San Jose Juan Santamaria...",90.718400,200.0,"The segment is a short 5.1‑minute cruise at 38,000 ft on an A320ne...",✔️ [-11942.468],NaN
3,"{'aircraft_type': 'A20N', 'origin_name': 'Montreal Pierre Elliott ...",74.389088,190.0,The segment lasts about 5.03 minutes (≈0.0838 h). The aircraft is ...,✔️ [-13365.883],NaN
4,"{'aircraft_type': 'B738', 'origin_name': 'Istanbul Sabiha Gokcen I...",526.166720,400.0,The segment is a cruise phase lasting 9.423 minutes (0.157 hours) ...,✔️ [-15918.041],NaN
...,...,...,...,...,...,...
395,"{'aircraft_type': 'B789', 'origin_name': 'Shanghai Pudong Internat...",400.068144,350.0,The segment is a short cruise leg lasting 3.948 minutes (~0.0658 h...,✔️ [-2506.819],NaN
396,"{'aircraft_type': 'A20N', 'origin_name': 'Guatemala City La Aurora...",90.718400,223.4,The segment lasts 5.362 minutes (about 0.0894 hours). For an Airbu...,✔️ [-17604.407],NaN
397,"{'aircraft_type': 'A359', 'origin_name': 'Taipei Taoyuan Internati...",1769.008800,1240.0,The segment is a short cruise leg of about 15 minutes (0.25 h) at ...,✔️ [-279850.310],NaN
398,"{'aircraft_type': 'A359', 'origin_name': 'Rome Leonardo da Vinci F...",1700.000000,1800.0,"The segment shows a 20‑minute level flight at cruise altitude (38,...",✔️ [-10000.000],NaN


In [49]:
def compute_rmse(eval_results):
    rmse = math.sqrt(abs(eval_results.score)/len(eval_results.results))
    return rmse

In [50]:
compute_rmse(eval_results)

232.64432273322294

In [ ]:
# dump in a csv file
import pandas as pd
import csv

results_list = []
for result in eval_results.results:
    true_value = result[0].fuel_burnt
    reasoning = result[1].reasoning
    pred_value = result[1].fuel_burnt
    metric = result[2]
    results_list.append({
        "true_value": true_value,
        "reasoning": reasoning,
        "pred_value": pred_value,
        "metric": metric
    })
results_df = pd.DataFrame(results_list)
results_df.to_csv("eval_results.csv", index=False, quoting=csv.QUOTE_ALL)

## Batch API

In [ ]:
# 0. prepare batches of 100 examples each


In [ ]:


# 1. create a jsonl file
artifacts = fuel_cot.create_batch_file(
    examples,
    input_file_path="batches/math_input.jsonl",
)
artifacts

## Usage

In [51]:
for lm in [gpt_oss_120b]:
  cost = sum([x['cost'] for x in lm.history if x['cost'] is not None])
  print(cost)

0.3628254
